In [2]:
import rasterio
from rasterio.plot import show
import numpy as np
from skimage.filters import threshold_otsu

def extract_water_bodies(image_path, water_threshold=0.1):
    # Open the satellite image
    with rasterio.open(image_path) as src:
        # Read the image bands
        red = src.read(3)
        green = src.read(2)
        blue = src.read(1)

    # Convert bands to float and normalize
    red = red.astype('float32') / 255
    green = green.astype('float32') / 255
    blue = blue.astype('float32') / 255

    # Calculate NDWI (Normalized Difference Water Index)
    ndwi = (green - red) / (green + red)

    # Apply thresholding to extract water bodies
    water_mask = ndwi > water_threshold

    return water_mask

# Specify the path to the satellite image
image_path = "/Users/gaurabsapkota/Downloads/Water Bodies Dataset/Images/water_body_19.jpg"

# Extract water bodies from the image
water_bodies = extract_water_bodies(image_path)

# Save the water bodies mask as a new image
with rasterio.open(image_path) as src:
    profile = src.profile
    profile.update(dtype=rasterio.uint8, count=1)

    with rasterio.open("water_bodies_mask.tif", 'w', **profile) as dst:
        dst.write(water_bodies.astype(rasterio.uint8), 1)

# Display the water bodies mask
show(water_bodies, cmap='viridis')


ModuleNotFoundError: No module named 'rasterio'

In [ ]:
import rasterio
import numpy as np

# Step 1: Open the satellite image
with rasterio.open('satellite_image.tif') as src:
    # Step 2: Read the satellite image
    img = src.read()
    # Step 3: Extract metadata
    crs = src.crs
    transform = src.transform
    height = src.height
    width = src.width
    extent = src.bounds
    
    # Step 4: Define mask criteria
    # For example, to create a mask for all pixels with a certain range of NDVI values:
    # NDVI (Normalized Difference Vegetation Index) calculation:
    # ndvi = (band4 - band3) / (band4 + band3)
    # Assuming band4 is NIR (Near Infrared) and band3 is Red
    band4 = img[3]  # Assuming band 4 is NIR
    band3 = img[2]  # Assuming band 3 is Red
    ndvi = (band4 - band3) / (band4 + band3)
    
    # Define the threshold for NDVI to create the mask
    ndvi_threshold = 0.5
    mask = np.where(ndvi > ndvi_threshold, 1, 0)  # Create binary mask
    
    # Step 5: Create the mask
    
    # Step 6: Write the mask to a new file
    # Set metadata for the new raster file
    meta = src.meta.copy()
    meta.update(dtype=rasterio.uint8, count=1)  # Update metadata for single-band mask
    
    # Write the mask to a new raster file
    with rasterio.open('mask.tif', 'w', **meta) as dst:
        dst.write(mask.astype(rasterio.uint8), 1)
